In [0]:
from pyspark.sql.functions import col
from silver_tbl_columns_dict import silver_tbl_dict

weather_df = spark.table('`nyc_taxis_weather`.bronze_nyc_taxis.b_weather_oct_dec_2024').drop(col("_rescued_data"))
taxis_df = spark.table('`nyc_taxis_weather`.bronze_nyc_taxis.b_nyc_taxis_oct_dec_2024').drop(col("_rescued_data"))
lookup_df = spark.table('`nyc_taxis_weather`.bronze_nyc_taxis.b_taxi_zone_lookup').drop(col("_rescued_data"))
#weather codes .csv table uploaded manually (reference: https://dev.meteostat.net/formats.html#weather-condition-codes)
w_code_df = spark.table('`nyc_taxis_weather`.bronze_nyc_taxis.b_weather_codes').drop(col("_rescued_data"))

In [0]:
from pyspark.sql.functions import col, date_trunc, round

#Truncating pickup time to enable joining weather data (weather data available for full hours)
dt_taxis_hour = taxis_df.withColumn("pickup_hour", date_trunc("hour", col("tpep_pickup_datetime")))

#Joining weather data to taxi data by hour truncated above
dt_taxis_weather = dt_taxis_hour.join(weather_df, dt_taxis_hour.pickup_hour == weather_df.time, "left").drop(col("datetime"), col("time"), col("pickup_hour"))

#Joining dictionary table to create final result table ready to be analysed
result_table = dt_taxis_weather.join(lookup_df.withColumnRenamed("LocationID", "PULocationID_tmp")
                                    .withColumnRenamed("Borough", "PUBorough").drop(col("service_zone"))
                                    .withColumnRenamed("Zone", "PUZone"),dt_taxis_weather.PULocationID == col("PULocationID_tmp"),"left").join(lookup_df.withColumnRenamed("LocationID", "DOLocationID_tmp")
                                    .withColumnRenamed("Borough", "DOBorough")
                                    .withColumnRenamed("Zone", "DOZone"),dt_taxis_weather.DOLocationID == col("DOLocationID_tmp"),"left").join(w_code_df,dt_taxis_weather.coco == col("w_code"), "left")
                                    
#Adding fare duration and dropping unnecessary post-join columns                                   
result_table = result_table.withColumn("trip_duration", round((col("tpep_dropoff_datetime").cast("long") - col("tpep_pickup_datetime").cast("long")) / 60, 2)).drop("PULocationID_tmp", "DOLocationID_tmp", "w_code", "coco", "store_and_fwd_flag", "PULocationID", "DOLocationID", "congestion_surcharge")

#Renaming columns for clarity
result_table = result_table.withColumnsRenamed(silver_tbl_dict)

In [0]:
from pyspark.sql.functions import to_date
#saving table with overwrite and mergeSchema which allows schema evolution, in latest version of table schema was updated along with the partitioning by pickup day, few columns were renamed for better naming clarity

result_table.withColumn("pickup_date", to_date("pickup_datetime")).write.partitionBy("pickup_date").option("mergeSchema", "true").option("overwriteSchema", "true").mode("overwrite").saveAsTable("`nyc_taxis_weather`.silver_nyc_taxis.s_nyc_taxis_weather_oct_dec")